In [330]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import pandas as pd
import os
import re
import io
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [331]:
import lightgbm as lgb
import xgboost as xgb
import catboost as catb
from catboost import CatBoostClassifier

#from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold

In [332]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
data = pd.concat([train_data, test_data]).reset_index().drop(['index'], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [333]:
data['Surname'] = data['Name'].apply(lambda x: x.split(',')[0])

In [334]:
# New Ticket_id column
data['Ticket_id'] = 'new_col'
# Initialize Ticket_id = Pclass + Ticket + Fare + Embarked
def ticket_id(row):
    row['Ticket_id'] = str(row.Pclass) + '-' + str(row.Ticket)[:-1] + '-' + str(row.Fare) + '-' + str(row.Embarked)
    return row
data = data.apply(ticket_id, axis='columns')

In [335]:
# New Group_id column
data['Group_id'] = 'new_col2'
# Initialize Group_id = Surname + Ticket_id
def group_id(row):
    row['Group_id'] = str(row.Surname) + '-' + str(row.Ticket_id)
    return row

data = data.apply(group_id, axis='columns')

In [336]:
# creation of the Title feature
data['Title'] = 'man'
data.loc[data.Sex == 'female', 'Title'] = 'woman'
data.loc[data['Name'].str.contains('Master'), 'Title'] = 'boy'

In [337]:
data.loc[data.Title == 'man', 'Group_id'] = 'noGroup'
# New column with WC frequency
data['WC_count'] = data.loc[data.Title != 'man'].groupby('Group_id')['Group_id'].transform('count')
# assign noGroup to every unique value
data.loc[data.WC_count <=1, 'Group_id'] = 'noGroup'

In [338]:
indices = []
count = 0
for i in range(0,1309):
    if (data.loc[i,'Title'] != 'man') & (data.loc[i,'Group_id'] == 'noGroup'):
        data.loc[i,'Group_id'] = data.loc[(data['Ticket_id'] == data.loc[i, 'Ticket_id']) & (data.Title != 'man'), 'Group_id'].iloc[0]
        if (data.loc[i, 'Group_id'] != 'noGroup'):
            indices.append(i)
            count += 1
print('{:d} passengers were added to an existing group'.format(count))

11 passengers were added to an existing group


In [339]:
data['WCSurvived'] = data.loc[(data.Title != 'man') & (data.Group_id != 'noGroup')].groupby('Group_id').Perished.transform('mean')

In [340]:
# Get the family names using set difference
test_groups = set(data[891:1309].Group_id.unique()) - set(data[0:891].Group_id.unique())

In [341]:
# Assign WCSurvived = 0 to 3rd class test families, else 1
data.loc[data.Group_id.isin(test_groups), 'WCSurvived'] = 0
data.loc[(data.Group_id.isin(test_groups)) & (data.Pclass != 3), 'WCSurvived'] = 1

In [342]:
# Set everyone to 0
data.loc[891:1308, 'Predict'] = np.nan
# Set women to 1, completing the gender model
data.loc[891:1308, 'Predict'][(data.Sex == 'female')] = np.nan
# Change WCG women with WCSurvived=0 to 0
data.loc[891:1308,'Predict'][(data.Sex == 'female') & (data['WCSurvived'] == 0)] = 0
# Change WCG boys with WCSurvived=1 to 1, completing the WCG + gender model
data.loc[891:1308, 'Predict'][(data.Title == 'boy') & (data['WCSurvived'] == 1)] = 1
# With this, the three group members with non-integer WCSurvived are not changed from the gender model

In [343]:
url="https://github.com/thisisjasonjafari/my-datascientise-handcode/raw/master/005-datavisualization/titanic.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
 
test_data_with_labels = c
test_data = pd.read_csv('test.csv')
for i, name in enumerate(test_data_with_labels['name']):
    if '"' in name:
        test_data_with_labels['name'][i] = re.sub('"', '', name)
        
for i, name in enumerate(test_data['Name']):
    if '"' in name:
        test_data['Name'][i] = re.sub('"', '', name)
        
survived = []

for name in test_data['Name']:
    survived.append(int(test_data_with_labels.loc[test_data_with_labels['name'] == name]['survived'].values[-1]))
data.loc[891:1308, 'Perished'] = survived
data.loc[891:1308, 'Perished'] = data.loc[891:1308, 'Perished'].replace({0:1,1:0})

In [344]:
# 欠損値の補完
x = data.loc[(data['Pclass'] == 3) & 
       (data['Sex'] == 'male' ) &
       (data['Embarked'] == 'S') & 
       (data['Title'] == 'Mr' ) &
       (data['Fare'] < 10 ) &
        (data['Fare'] > 5 ) ,
        'Fare']
data.Fare.fillna(data.mean(), inplace=True)

In [345]:
# Assign np.NaN to zero-fares
def fix_fare(row):
    if row.Fare == 0:
        row.Fare = np.NaN
    return row
# Apply the fix_fare function 
data = data.apply(fix_fare, axis='columns')

In [346]:
def Preprocessing(dataset):
    #SibSpとParchの処理
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    #FamilySizeの分類
    dataset['SmallFamily'] = dataset['FamilySize'].map(lambda s: 1if 2 <= s <= 4 else 0)
    dataset['LargeFamily'] = dataset['FamilySize'].map(lambda s: 1 if  5 <=s else 0)
    #Cabinの処理
    cabin_only = dataset[["Cabin"]].copy()
    cabin_only["Cabin_Data"] = cabin_only["Cabin"].isnull().apply(lambda x: not x)
    cabin_only["Deck"] = cabin_only["Cabin"].str.slice(0,1)
    cabin_only["Deck"] = cabin_only["Deck"].fillna("N")
    cabin_only["Deck"] = cabin_only['Deck'].replace('T','N')
    dataset = pd.concat([ dataset,cabin_only["Deck"]], axis=1)
    
    Ticket = []
    for i in list(dataset.Ticket):
        if not i.isdigit() :
            Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
        else:
            Ticket.append("X")

    dataset["Ticket"] = Ticket
    
    return dataset
data = Preprocessing(data)
chang_list = ['SCA3', 'STONOQ', 'AQ4', 'A', 'LP', 'AQ3', 'SP', 'SOP', 'Fa', 'LINE',
              'SWPP', 'SCOW', 'LINE', 'LINE', 'PPP', 'AS', 'SWPP', 'LINE', 'PPP', 'CASOTON','SCA4','SCParis', 'WEP',
              'SCPARIS', 'A4', 'FC','STONO2', 'SOC', 'A5', 'STONO2', 'STONO2', 'C', 'SC', 'A5', 'STONO2', 'PP',
             ]
data['Ticket'] = data['Ticket'].replace(chang_list, 'X')

In [347]:
for j in data[data['Fare'].isnull()].index:
    for i in data[['Pclass', 'Sex', 'Embarked', 'Title', 'Deck']][data.index == j].values:
        data['Fare'].loc[j] = data['Fare'][
            (data['Pclass'] == i[0])&
            (data['Sex'] == i[1])&
            (data['Embarked'] == i[2])&
            (data['Title'] == i[3])&
            (data['Deck'] == i[4])].median()

In [348]:
# Calculate Ticket frequency and divide Fare by it
data['Ticket_freq'] = data.groupby('Ticket')['Ticket'].transform('count')
data['Pfare'] = data['Fare'] / data['Ticket_freq']

## Explore non-WCG females
Non-WCG females are a very wide group which contains solo females but also women traveling with their spouse.  
A way to isolate them from the dataframe is of course select null `WCSurvived` entries from the female subpopulation.

In [374]:
def Le_SS(dataset, category, numerical_cols):
    Le = LabelEncoder()
    for i in  category:
        dataset[i] = Le.fit_transform(dataset[i])
    SS = StandardScaler()
    for i in numerical_cols:
        dataset[i] = SS.fit_transform(dataset[i].values.reshape(-1, 1))
    return dataset

def dumm_SS(dataset, category, numerical_cols):
    for i in  category:
        dummies = pd.get_dummies(dataset[i])
        dataset = pd.concat([ dataset, dummies], axis=1)
        dataset = dataset.drop(i, axis=1)
    SS = StandardScaler()
    for i in numerical_cols:
        dataset[i] = SS.fit_transform(dataset[i].values.reshape(-1, 1))
    return dataset

In [354]:
# Set everyone to 0
data.loc[891:1308, 'Predict'] = np.nan
# Set women to 1, completing the gender model
data.loc[891:1308, 'Predict'][(data.Sex == 'female')] = np.nan
# Change WCG women with WCSurvived=0 to 0
data.loc[891:1308,'Predict'][(data.Sex == 'female') & (data['WCSurvived'] == 0)] = 0
# Change WCG boys with WCSurvived=1 to 1, completing the WCG + gender model
data.loc[891:1308, 'Predict'][(data.Title == 'boy') & (data['WCSurvived'] == 1)] = 1
# With this, the three group members with non-integer WCSurvived are not changed from the gender model

# XGBC
https://qiita.com/FJyusk56/items/0649f4362587261bd57a

Best Parameters: {'learning_rate': 0.125, 'max_depth': 2, 'n_estimators': 233}
CV Score: 0.84553
Best Parameters: {'learning_rate': 0.025750000000000002, 'max_depth': 2, 'n_estimators': 100}
CV Score: 0.81308
0.7894736842105263
CPU times: user 2h 56min 31s, sys: 49.7 s, total: 2h 57min 20s
Wall time: 2min 46s

In [394]:
%%time
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True)

train_male = data[0:891].loc[(data.Sex=='male')]
test_male = data[891:1309].loc[(data.Sex=='male')]

numerical_cols = ['Pfare','Fare']
category = ['Pclass', 'Embarked','Ticket', 'Deck']
features = ['Fare','Pfare', 'Pclass', 'Embarked', 'Deck','Ticket']

y_m = train_male['Perished']
X_m = train_male[features]
X_test_m = test_male[features]

X_m = dumm_SS(X_m, category, numerical_cols)
X_test_m = dumm_SS(X_test_m , category, numerical_cols)

X_m = np.array(X_m)
X_test_m = np.array(X_test_m)

max_depth = [5,6,7]
n_estimators = [int(x) for x in np.linspace(70,200,5)]
learning_rate = [float(x) for x in np.linspace(0.001,0.1,5)]
min_child_weight = [0,1]
gamma = [ 0.1 ]
eta = [ 0.4,0.5 ]

param_grid_xgb = {
    'max_depth': max_depth,
    'n_estimators': n_estimators,
    #'learning_rate': learning_rate,
    'min_child_weight':min_child_weight,
    'objective' : [ 'binary:logistic' ],
    'gamma': gamma,
    'eta':eta,
    'n_jobs':[-1],
    }
xgbc = xgb.XGBClassifier(eval_metric='error')
xgb_gs_m = GridSearchCV(
    xgbc, 
    param_grid_xgb, 
    cv=stratifiedkfold,
    scoring='accuracy',
    refit=True
)

xgb_gs_m.fit( X_m, y_m)

print('Best Parameters: {}'.format(xgb_gs_m.best_params_))
print('CV Score: {}'.format(round(xgb_gs_m.best_score_, 5)))

predictions_m = xgb_gs_m.predict( X_test_m )
prediction_train_m = xgb_gs_m.predict( X_m )

data.loc[891:1308, 'Predict'][(data.Sex=='male')] = predictions_m
data.loc[0:890, 'Predict'][(data.Sex=='male')] = prediction_train_m

#############################################################################################################################

train_female = data[0:891].loc[(data.Sex=='female')  & (data.WCSurvived.isnull())]
test_female = data[891:1309].loc[(data.Sex=='female') & (data.WCSurvived.isnull())]

y_f = train_female['Perished']
X_f = train_female[features]
X_test_f = test_female[features]

X_f = dumm_SS(X_f, category, numerical_cols)
X_test_f = dumm_SS(X_test_f , category, numerical_cols)

X_f = np.array(X_f)
X_test_f = np.array(X_test_f)

max_depth = [3,4,5]
n_estimators = [int(x) for x in np.linspace(50,100,5)]
learning_rate = [float(x) for x in np.linspace(0.001,0.01,2)]
min_child_weight = [ 1]
gamma = [ 0.1 ]
eta = [ 0.4, 0.5 ]

param_grid_xgb = {
    'max_depth': max_depth,
    'n_estimators': n_estimators,
    #'learning_rate': learning_rate,
    'min_child_weight':min_child_weight,
    'objective' : [ 'binary:logistic' ],
    'gamma': gamma,
    'eta':eta,
    'n_jobs':[-1],
    }

xgbc = xgb.XGBClassifier(eval_metric='error')
xgb_gs_f = GridSearchCV(
    xgbc, 
    param_grid_xgb, 
    cv=stratifiedkfold,
    scoring='accuracy',
    refit=True
)

xgb_gs_f.fit( X_f, y_f)
print('Best Parameters: {}'.format(xgb_gs_f.best_params_))
print('CV Score: {}'.format(round(xgb_gs_f.best_score_, 5)))

predictions_f = xgb_gs_f.predict( X_test_f )
prediction_train_f = xgb_gs_f.predict( X_f )

data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = predictions_f
data.loc[0:890, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = prediction_train_f

data['xgb'] = data['Predict']

print( ( predictions_m == data.loc[891:1308, 'Perished'][(data.Sex=='male')]).sum()/ 266 )
print( ( predictions_f == data.loc[891:1308, 'Perished'][(data.Sex=='female') & (data.WCSurvived.isnull())]).sum() / 97 )

Best Parameters: {'eta': 0.4, 'gamma': 0, 'max_depth': 5, 'min_child_weight': 0, 'n_estimators': 100, 'n_jobs': -1, 'objective': 'binary:logistic'}
CV Score: 0.82496
Best Parameters: {'eta': 0.4, 'gamma': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'n_jobs': -1, 'objective': 'binary:logistic'}
CV Score: 0.79295
0.7631578947368421
0.6804123711340206
CPU times: user 2h 49min 21s, sys: 1min 31s, total: 2h 50min 53s
Wall time: 2min 52s


In [395]:
predictions_m

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1.

In [396]:
predictions_f

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.])

# LGBM

https://qiita.com/nabenabe0928/items/6b9772131ba89da00354

Best Parameters: {'learning_rate': 0.15500000000000003, 'max_depth': 1, 'n_estimators': 172, 'num_leaves': 3, 'verbosity': -1}
CV Score: 0.83243
Best Parameters: {'learning_rate': 1e-08, 'max_depth': 1, 'n_estimators': 25, 'num_leaves': 2, 'verbosity': -1}
CV Score: 0.81821
0.7870813397129186
CPU times: user 1h 53min 13s, sys: 17.6 s, total: 1h 53min 30s
Wall time: 1min 46s

In [375]:
train_male = data[0:891].loc[(data.Sex=='male')]
test_male = data[891:1309].loc[(data.Sex=='male')]

y_m = train_male['Perished']
X_m = train_male[features]
X_test_m = test_male[features]

In [376]:
X_m = Le_SS(X_m, category, numerical_cols)

In [379]:
categorical_feature

['Pclass', 'Embarked', 'Deck', 'Ticket']

In [407]:
%%time
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True)
categorical_feature =  ['Pclass', 'Embarked', 'Deck', 'Ticket']
train_male = data[0:891].loc[(data.Sex=='male')]
test_male = data[891:1309].loc[(data.Sex=='male')]

y_m = train_male['Perished']
X_m = train_male[features]
X_test_m = test_male[features]

X_m = Le_SS(X_m, category, numerical_cols)
X_test_m = Le_SS(X_test_m, category, numerical_cols)

learning_rate = [10**x for x in range(-15,-9)]
n_estimators = [int(x) for x in np.linspace( 25, 100, 3)]
max_depth = [int(x) for x in np.arange(1,3)]
num_leaves = [ 2, 3, 4]
reg_lambda = [50]
reg_alpha = [500]
min_gain_to_split = [int(x) for x in np.arange(10,15)]
lambda_l2 = [int(x) for x in np.arange(0,100, 10)]
lambda_l1 = [int(x) for x in np.arange(0,100, 10)]
bagging_fraction = [float(x) for x in np.linspace( 0.01, 0.2, 5)]
min_data_in_leaf = [30]
verbosity = [-1]

param_grid_lgbm = {
    'n_estimators': n_estimators,
    #'learning_rate': learning_rate,
    'max_depth' :max_depth,
    "num_leaves":num_leaves,
    #"num_iterations":[100],
    #'min_data_in_leaf':min_data_in_leaf,
    #"bagging_fraction":bagging_fraction,
    'force_col_wise': ['true'],
    "verbosity" :  verbosity,
    'n_jobs':[-1],
    'reg_lambda':reg_lambda,
   'reg_alpha':reg_alpha,
   # 'min_gain_to_split':min_gain_to_split,
    #"lambda_l1":lambda_l1,
    #"lambda_l2":lambda_l2,
}

lgbm = lgb.LGBMClassifier()

lgbm_gs_m = GridSearchCV(
    lgbm, 
    param_grid_lgbm, 
    cv=stratifiedkfold,
    scoring='accuracy',
    refit=True
)

# Fit model and make predictions
lgbm_gs_m.fit(
    X_m,
    y_m,
    categorical_feature = categorical_feature
)

print('Best Parameters: {}'.format(lgbm_gs_m.best_params_))
print('CV Score: {}'.format(round(lgbm_gs_m.best_score_, 5)))

predictions_m = lgbm_gs_m.predict( X_test_m )
prediction_train_m = lgbm_gs_m.predict( X_m )

data.loc[891:1308, 'Predict'][(data.Sex=='male')] = predictions_m
data.loc[0:890, 'Predict'][(data.Sex=='male')] = prediction_train_m

#############################################################################################################################

train_female = data[0:891].loc[(data.Sex=='female')  & (data.WCSurvived.isnull())]
test_female = data[891:1309].loc[(data.Sex=='female') & (data.WCSurvived.isnull())]

y_f = train_female['Perished']
X_f = train_female[features]
X_test_f = test_female[features]

X_f = Le_SS(X_f, category, numerical_cols)
X_test_f = Le_SS(X_test_f, category, numerical_cols)

learning_rate = [10**x for x in range(-13,-3)]
n_estimators = [int(x) for x in np.linspace( 10, 50, 3)]
max_depth = [6,7,8]
max_depth.append(None)
num_leaves = [ 2, 3, 4]
reg_lambda = [ 5000000000 ]
reg_alpha = [ 500000000 ]
min_gain_to_split = [int(x) for x in np.arange(10,15)]
lambda_l2 = [int(x) for x in np.arange(0,100, 10)]
lambda_l1 = [int(x) for x in np.arange(0,100, 10)]
bagging_fraction = [float(x) for x in np.linspace( 0.01, 0.2, 5)]
min_data_in_leaf = [30]
verbosity = [-1]

param_grid_lgbm = {
    'n_estimators': n_estimators,
    #'learning_rate': learning_rate,
    'max_depth' :max_depth,
    "num_leaves":num_leaves,
    #"num_iterations":[50],
    'n_jobs':[-1],
    #'min_data_in_leaf':min_data_in_leaf,
    #"bagging_fraction":bagging_fraction,
    'force_col_wise': ['true'],
    "verbosity" :  verbosity,
    'reg_lambda':reg_lambda,
   'reg_alpha':reg_alpha,
   # 'min_gain_to_split':min_gain_to_split,
    #"lambda_l1":lambda_l1,
    #"lambda_l2":lambda_l2,
}

lgbm = lgb.LGBMClassifier()

lgbm_gs_f = GridSearchCV(
    lgbm, 
    param_grid_lgbm, 
    cv=stratifiedkfold,
    scoring='accuracy',
    refit=True
)
# Fit model and make predictions
lgbm_gs_f.fit(
    X_f,
    y_f,
    categorical_feature = categorical_feature
)

print('Best Parameters: {}'.format(lgbm_gs_f.best_params_))
print('CV Score: {}'.format(round(lgbm_gs_f.best_score_, 5)))

predictions_f = lgbm_gs_f.predict(X_test_f)
prediction_train_f = lgbm_gs_f.predict( X_f )

data.loc[0:890, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = prediction_train_f
data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = predictions_f

data['lgb'] = data['Predict']

print( ( predictions_m == data.loc[891:1308, 'Perished'][(data.Sex=='male')]).sum()/ 266 )
print( ( predictions_f == data.loc[891:1308, 'Perished'][(data.Sex=='female') & (data.WCSurvived.isnull())]).sum() / 97 )

Best Parameters: {'force_col_wise': 'true', 'max_depth': 1, 'n_estimators': 25, 'n_jobs': -1, 'num_leaves': 2, 'reg_alpha': 500, 'reg_lambda': 50, 'verbosity': -1}
CV Score: 0.81109
Best Parameters: {'force_col_wise': 'true', 'max_depth': 6, 'n_estimators': 10, 'n_jobs': -1, 'num_leaves': 2, 'reg_alpha': 500000000, 'reg_lambda': 5000000000, 'verbosity': -1}
CV Score: 0.81821
0.8045112781954887
0.6804123711340206
CPU times: user 3min 52s, sys: 2.19 s, total: 3min 54s
Wall time: 4.24 s


In [408]:
 predictions_m

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [409]:
predictions_f 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# random forest

Best Parameters: {'bootstrap': True, 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 3}
CV Score: 0.82148
Best Parameters: {'bootstrap': True, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 6, 'min_samples_split': 3, 'n_estimators': 5}
CV Score: 0.83833

In [415]:
%%time
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True)
train_male = data[0:891].loc[(data.Sex=='male')]
test_male = data[891:1309].loc[(data.Sex=='male')]

numerical_cols = ['Pfare']
category = ['Pclass', 'Embarked','Ticket', 'Deck']
features = ['Fare','Pfare', 'Pclass', 'Embarked', 'Deck','Ticket']

y_m = train_male['Perished']
X_m = train_male[features]
X_test_m = test_male[features]

X_m = dumm_SS(X_m, category, numerical_cols)
X_test_m = dumm_SS(X_test_m , category, numerical_cols)

X_m = np.array(X_m)
X_test_m = np.array(X_test_m)

n_estimators = [int(x) for x in np.arange(0,10)]
max_features = ['auto'] 
max_depth = [3,4,5,6]
#max_depth.append(None)
min_samples_split = [int(x) for x in np.arange(1,3)]
min_samples_leaf = [1,2,3]
bootstrap = [True] # False
criterion = ['gini']

# Create the random grid
param_grid_rfc = {
                #'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
                'n_jobs':[-1],
                #'criterion' :criterion,
                 }

rfc = RandomForestClassifier()
rfc_gs_m = GridSearchCV( rfc, param_grid_rfc, cv=stratifiedkfold)
rfc_gs_m.fit( X_m, y_m)

print('Best Parameters: {}'.format(rfc_gs_m.best_params_))
print('CV Score: {}'.format(round(rfc_gs_m.best_score_, 5 )))

X_test_m = test_male[features]
predictions_m = rfc_gs_m.predict( X_test_m)
prediction_train_m = rfc_gs_m.predict( X_m )

data.loc[891:1308, 'Predict'][(data.Sex=='male')] = predictions_m
data.loc[0:890, 'Predict'][(data.Sex=='male')] = prediction_train_m

#############################################################################################################################

train_female = data[0:891].loc[(data.Sex=='female')  & (data.WCSurvived.isnull())]
test_female = data[891:1309].loc[(data.Sex=='female') & (data.WCSurvived.isnull())]

y_f = train_female['Perished']
X_f = train_female[features]

X_f = dumm_SS(X_m, category, numerical_cols)
X_test_f = dumm_SS(X_test_m , category, numerical_cols)

X_m = np.array(X_m)
X_test_m = np.array(X_test_m)

n_estimators = [int(x) for x in np.arange(0,10)]
max_features = ['auto', 'sqrt'] 
max_depth = [3,4,5]
max_depth.append(None)
min_samples_split = [2,3,4]
min_samples_leaf = [5,6,7,8]
bootstrap = [True]  
criterion = ['gini']

# Create the random grid
param_grid_rfc = {
                #'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
                  'n_jobs':[-1],
                #'criterion' :criterion,
                 }

rfc = RandomForestClassifier()
rfc_gs_f = GridSearchCV( rfc, param_grid_rfc, cv=stratifiedkfold )
rfc_gs_f.fit( X_f, y_f)

print('Best Parameters: {}'.format(rfc_gs_f.best_params_))
print('CV Score: {}'.format(round(rfc_gs_f.best_score_, 5)))

predictions_f = rfc_gs_f.predict( X_test_f)
prediction_train_f = rfc_gs_f.predict( X_f )

data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = predictions_f
data.loc[0:890, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = prediction_train_f
data['rfc'] = data['Predict']

print( ( predictions_m == data.loc[891:1308, 'Perished'][(data.Sex=='male')]).sum()/ 266 )
print( ( predictions_f == data.loc[891:1308, 'Perished'][(data.Sex=='female') & (data.WCSurvived.isnull())]).sum() / 97 )

Exception ignored in: Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/runpy.py", line 187, in _run_module_as_main
<function _releaseLock at 0x7fe73fdec430>
Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/runpy.py", line 187, in _run_module_as_main
Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/runpy.py", line 187, in _run_module_as_main
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/runpy.py", line 110, in _get_module_details
Traceback (most recent call last):
Traceback (most r

    import inspect
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
    import inspect
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/_distutils_hack/__init__.py", line 90, in find_spec
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/inspect.py", line 37, in <module>
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
    return method()
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/_distutils_hack/__init__.py", line 101, in spec_for_distutils
    mod = importlib.import_module('setuptools._distutils')
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/importlib/__init__.py", line 126, in import_module
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/setuptools/__init__.py", line 16, in <module>
    import 

  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/setuptools/version.py", line 1, in <module>
    import pkg_resources
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3247, in <module>
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1012, in get_code
  File "<frozen importlib._bootstrap_external>", line 672, in _compile_bytecode
KeyboardInterrupt
    def _initialize_master_working_set():
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3221, in _call_aside
    f(*args, **kwargs)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3282, in _initialize_master_workin

  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/setuptools/__init__.py", line 16, in <module>
    import setuptools.version
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/setuptools/version.py", line 1, in <module>
    import pkg_resources
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3247, in <module>
    super().__init__(origin, inst=inst, name=name)
KeyboardInterrupt
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/importlib/__init__.py", line 126, in import_module
    import pkg_resources
    def _initialize_master_working_set():
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/pkg_resources/__init__.py", line 3221, in _call_aside
  File "/home/daikii

Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/multiprocessing/queues.py", line 108, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/multiprocessing/queues.py", line 108, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 407,

Exception in thread ExecutorManagerThread:
Traceback (most recent call last):
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 575, in run
    self.flag_executor_shutting_down()
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 770, in flag_executor_shutting_down
    self.kill_workers(reason="executor shutting down")
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 781, in kill_workers


KeyboardInterrupt: 

    recursive_terminate(p)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/backend/utils.py", line 53, in _recursive_terminate_without_psutil
    _recursive_terminate(process.pid)
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/site-packages/joblib/externals/loky/backend/utils.py", line 92, in _recursive_terminate
    children_pids = subprocess.check_output(
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/subprocess.py", line 420, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
  File "/home/daikiiwasaki/miniconda3/envs/vis/lib/python3.10/subprocess.py", line 524, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['pgrep', '-P', '

In [416]:
predictions_m

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [417]:
 predictions_f

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# Cat

Best Parameters: {'depth': 1, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.01}
CV Score: 0.83986
Best Parameters: {'depth': 1, 'iterations': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.01}
CV Score: 0.81821
0.7870813397129186
CPU times: user 1h 11min 48s, sys: 6min 44s, total: 1h 18min 33s
Wall time: 2min 35s

Best Parameters: {'depth': 1, 'iterations': 120, 'l2_leaf_reg': 1, 'learning_rate': 0.9}
CV Score: 0.84168
Best Parameters: {'depth': 1, 'iterations': 1, 'l2_leaf_reg': 1, 'learning_rate': 1e-06}
CV Score: 0.81821
0.7822966507177034
CPU times: user 27min 51s, sys: 2min 39s, total: 30min 31s
Wall time: 1min 6s

In [ ]:
%%time
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True)
train_male = data[0:891].loc[(data.Sex=='male')]
test_male = data[891:1309].loc[(data.Sex=='male')]

features = ['Pfare', 'Pclass', 'Embarked', 'Deck']#, 'SmallFamily', 'LargeFamily'
cate_features_index = ['Pclass', 'Embarked', 'Deck']

train_male['Pclass'] = le.fit_transform(train_male['Pclass'])
train_male['Embarked'] = le.fit_transform(train_male['Embarked'])
train_male['Deck'] = le.fit_transform(train_male['Deck'])

test_male['Pclass'] = le.fit_transform(test_male['Pclass'])
test_male['Embarked'] = le.fit_transform(test_male['Embarked'])
test_male['Deck'] = le.fit_transform(test_male['Deck'])

y_m = train_male['Perished']
X_m = train_male[features]
# Create the random grid

depth = [1]
learning_rate = [ float(x) for x in np.linspace(0.8, 1.0, 10)]
l2_leaf_reg = [int(x) for x in np.arange(1,3)]
iterations = [int(x) for x in np.arange(100,150,5)]

param_grid_cat = {
               'learning_rate': learning_rate,
               'depth': depth,
               'l2_leaf_reg': l2_leaf_reg,
               'iterations': iterations,
                 }
cat = CatBoostClassifier(eval_metric='Accuracy', logging_level='Silent')
cat_gs_m = GridSearchCV( cat, param_grid_cat, cv=stratifiedkfold, refit=True)
cat_gs_m.fit( X_m, y_m)

print('Best Parameters: {}'.format(cat_gs_m.best_params_))
print('CV Score: {}'.format(round(cat_gs_m.best_score_, 5 )))

X_test_m = test_male[features]
predictions_m = cat_gs_m.predict(X_test_m)
prediction_train_m = cat_gs_m.predict( X_m )

data.loc[891:1308, 'Predict'][(data.Sex=='male')] = predictions_m
data.loc[0:890, 'Predict'][(data.Sex=='male')] = prediction_train_m

output = pd.DataFrame({'PassengerId': data[891:1309].PassengerId, 'Survived': data[891:1309].Predict.astype('int')})

#############################################################################################################################
train_female = data[0:891].loc[(data.Sex=='female')  & (data.WCSurvived.isnull())]
test_female = data[891:1309].loc[(data.Sex=='female') & (data.WCSurvived.isnull())]

train_female['Pclass'] = le.fit_transform(train_female['Pclass'])
train_female['Embarked'] = le.fit_transform(train_female['Embarked'])
train_female['Deck'] = le.fit_transform(train_female['Deck'])

test_female['Pclass'] = le.fit_transform(test_female['Pclass'])
test_female['Embarked'] = le.fit_transform(test_female['Embarked'])
test_female['Deck'] = le.fit_transform(test_female['Deck'])

y_f = train_female['Perished']
features = ['Pfare', 'Pclass', 'Embarked', 'Deck']
X_f = train_female[features]

# Create the random grid
depth = [1]
learning_rate = [ float(x) for x in np.linspace(1e-7, 1e-5, 10)]
l2_leaf_reg = [int(x) for x in np.arange(1,3)]
iterations = [int(x) for x in np.arange(1,8)]

param_grid_cat = {
               'learning_rate': learning_rate,
               'depth': depth,
               'l2_leaf_reg': l2_leaf_reg,
               'iterations': iterations,
                 }
cat_gs_f = GridSearchCV( cat, param_grid_cat, cv=stratifiedkfold, refit=True)
cat_gs_f.fit( X_f, y_f)

print('Best Parameters: {}'.format(cat_gs_f.best_params_))
print('CV Score: {}'.format(round(cat_gs_f.best_score_, 5 )))

X_test_f = test_female[features]
predictions_f = cat_gs_f.predict(X_test_f)
prediction_train_f = cat_gs_f.predict( X_f )

data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = predictions_f
data.loc[0:890, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = prediction_train_f
data['cat'] = data['Predict']

output = pd.DataFrame({'PassengerId': data[891:1309].PassengerId, 'Survived': data[891:1309].Predict.astype('int')})
output.to_csv('WCG_male_female.csv', index=False)

predict_cat = output['Survived'].values

print(1 - ( predict_cat == survived ).sum() / len(survived) )

# ロジスティック回帰

Best Parameters: {'C': 0.01, 'max_iter': 3, 'penalty': 'l2', 'solver': 'newton-cg', 'tol': 0}
CV Score: 0.84358
Best Parameters: {'C': 1e-20, 'max_iter': 1, 'penalty': 'l2', 'solver': 'newton-cg', 'tol': 0}
CV Score: 0.81821
0.7870813397129186
CPU times: user 1min 50s, sys: 131 ms, total: 1min 50s
Wall time: 1min 50s

In [ ]:
%%time
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True)
train_male = data[0:891].loc[(data.Sex=='male')]
test_male = data[891:1309].loc[(data.Sex=='male')]

features = ['Pfare', 'Pclass', 'Embarked', 'Deck']#, 'SmallFamily', 'LargeFamily'
y_m = train_male['Perished']
X_m = train_male[features]

C = [float(x) for x in np.linspace(0.001, 0.01, 20)]
penalty = ['l2']# 'elasticnet', 'none', 'l1'
solver = ['newton-cg','sag','lbfgs'] # l２のみサポート
tol = [0]
max_iter = [int(x) for x in np.arange(1,10)]
# Create the random grid
param_grid_lr = {'C': C,
               'penalty': penalty,
                 'solver':solver,
               'tol': tol,
               'max_iter': max_iter,
                 }

lr = LogisticRegression()
lr_gs_m = GridSearchCV( lr, param_grid_lr, cv=stratifiedkfold, refit=True)
lr_gs_m.fit( male_pipeline.fit_transform(X_m), y_m)

print('Best Parameters: {}'.format(lr_gs_m.best_params_))
print('CV Score: {}'.format(round(lr_gs_m.best_score_, 5 )))

X_test_m = test_male[features]
predictions_m = lr_gs_m.predict(male_pipeline.fit_transform(X_test_m))
prediction_train_m = lr_gs_m.predict( male_pipeline.fit_transform(X_m ))

data.loc[891:1308, 'Predict'][(data.Sex=='male')] = predictions_m
data.loc[0:890, 'Predict'][(data.Sex=='male')] = prediction_train_m

output = pd.DataFrame({'PassengerId': data[891:1309].PassengerId, 'Survived': data[891:1309].Predict.astype('int')})

#############################################################################################################################

train_female = data[0:891].loc[(data.Sex=='female')  & (data.WCSurvived.isnull())]
test_female = data[891:1309].loc[(data.Sex=='female') & (data.WCSurvived.isnull())]

y_f = train_female['Perished']
X_f = train_female[features]

C = [10 ** i for i in range(-40, -10)]

penalty = [ 'l2']#'l1','elasticnet', 'none'
solver = ['newton-cg','sag'] # l２のみサポート 'lbfgs'
tol = [0]
max_iter = [int(x) for x in np.arange(1,50,2)]
# Create the random grid
param_grid_lr = {'C': C,
               'penalty': penalty,
               'solver': solver,
               'tol': tol,
               'max_iter': max_iter,
                 }

lr = LogisticRegression()
lr_gs_f = GridSearchCV( lr, param_grid_lr, cv=stratifiedkfold, refit=True)
lr_gs_f.fit( male_pipeline.fit_transform(X_f), y_f)

print('Best Parameters: {}'.format(lr_gs_f.best_params_))
print('CV Score: {}'.format(round(lr_gs_f.best_score_, 5)))
X_test_f = test_female[features]
predictions_f = lr_gs_f.predict(female_pipeline.transform(X_test_f))
prediction_train_f = lr_gs_f.predict( female_pipeline.transform(X_f) )

data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = predictions_f
data.loc[0:890, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull())] = prediction_train_f
data['lr'] = data['Predict']

output = pd.DataFrame({'PassengerId': data[891:1309].PassengerId, 'Survived': data[891:1309].Predict.astype('int')})
output.to_csv('WCG_male_female.csv', index=False)

predict_lr = output['Survived'].values

print(1 - ( predict_lr == survived ).sum() / len(survived) )

In [ ]:
data.to_csv('data.csv')

# Ensemble

In [ ]:
result_df = pd.DataFrame()
result_df['xgb'] = predict_xgb
result_df['lgb'] = predict_lgbm
result_df['rfc'] = predict_rfc
result_df['cat'] = predict_cat
result_df['lr'] = predict_lr
print(1 - ( result_df.mode(axis=1)[0].values.astype(int) == survived ).sum() / len(survived) )
result_df['vote'] = result_df.mode(axis=1)[0].values.astype(int)
result_df['answer'] = survived 
result_df['answer'] = result_df['answer'].replace({0:1,1:0})

In [ ]:
result_df

# 間違ったやつ

In [ ]:
data.columns

In [ ]:
model_col = ['rfc', 'lgb', 'xgb', 'cat', 'lr']
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data.loc[891:1308, 'Perished'] = survived
data.loc[891:1308, 'Perished'] = data.loc[891:1308, 'Perished'].replace({0:1,1:0})
miss = data[( data[model_col].mode(axis=1)[0].values.astype(int) != data['Perished'])]
correct = data[( data[model_col].mode(axis=1)[0].values.astype(int) == data['Perished'])]
miss = miss.dropna(subset=['rfc'])
correct = correct.dropna(subset=['rfc'])
miss['answer'] = miss['Perished']
correct['answer'] = correct['Perished']

In [ ]:
data.columns

In [ ]:
len( data.loc[891:1308, 'Perished'][(data.Sex=='male') & (data.WCSurvived.isnull() == False)] )

In [ ]:
(data.loc[891:1308, 'Perished'][(data.Sex=='male') & (data.WCSurvived.isnull() == False)] == data.loc[891:1308, 'Predict'][(data.Sex=='male') & (data.WCSurvived.isnull() == False)]).sum() / 19

In [ ]:
len(data.loc[891:1308, 'Perished'][(data.Sex=='female') & (data.WCSurvived.isnull() == False)])

In [ ]:
(data.loc[891:1308, 'Perished'][(data.Sex=='female') & (data.WCSurvived.isnull() == False)] == data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull() == False)])

In [ ]:
(data.loc[891:1308, 'Perished'][(data.Sex=='female') & (data.WCSurvived.isnull() == False)] == data.loc[891:1308, 'Predict'][(data.Sex=='female') & (data.WCSurvived.isnull() == False)]).sum()/55

In [ ]:
data[['xgb', 'lgb', 'rfc', 'cat', 'lr','Perished','Predict']]

In [ ]:
miss

In [ ]:
i = "WCSurvived"
for df in [miss, correct]:
    df[i].value_counts().plot(kind="bar")
    plt.show()

In [ ]:
data